In [1]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
import os

In [2]:
mlflow.set_tracking_uri('https://mlflow-service-982318063078.us-west2.run.app')
mlflow.set_experiment('lab5-experiment')

2025/04/16 17:57:23 INFO mlflow.tracking.fluent: Experiment with name 'lab5-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1744851443790, experiment_id='2', last_update_time=1744851443790, lifecycle_stage='active', name='lab5-experiment', tags={}>

In [3]:
column_names = ['class', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 
                'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
                'proanthocyanins', 'color_intensity', 'hue', 
                'od280/od315_of_diluted_wines', 'proline']

df = pd.read_csv('data/wine/wine.data', header=None, names=column_names)

df.head()

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
df.describe()

,class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [5]:
y = df['class']
X = df.drop('class', axis=1)

In [6]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, shuffle=True)

In [7]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns,index=X_train.index)

X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns,index=X_val.index)

X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [8]:
os.makedirs('save_data_lab2', exist_ok=True)

X_train_scaled.to_parquet('save_data_lab2/x_train.parquet')
X_val_scaled.to_parquet('save_data_lab2/x_val.parquet')
X_test_scaled.to_parquet('save_data_lab2/x_test.parquet')

pd.DataFrame(y_train, columns=['class']).to_parquet('save_data_lab2/y_train.parquet')
pd.DataFrame(y_val, columns=['class']).to_parquet('save_data_lab2/y_val.parquet')
pd.DataFrame(y_test, columns=['class']).to_parquet('save_data_lab2/y_test.parquet')

In [15]:
def objective(params):
    classifier_type = params.pop("type")

    if classifier_type == "dt":
        clf = DecisionTreeClassifier(**params)
        model_name = "decision_tree_wine"
    elif classifier_type == "rf":
        clf = RandomForestClassifier(**params)
        model_name = "random_forest_wine"
    else:  # lr
        clf = LogisticRegression(**params, max_iter=2000)
        model_name = "logistic_regression_wine"

    with mlflow.start_run():
        acc = cross_val_score(clf, X_train_scaled, y_train, cv=5).mean()
        clf.fit(X_train_scaled, y_train)

        mlflow.set_tag("model_type", classifier_type)
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)

        mlflow.sklearn.log_model(
            clf, artifact_path="model",
            registered_model_name=model_name
        )

        return {"loss": -acc, "status": STATUS_OK}

In [16]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'dt',
        'criterion': hp.choice('dtree_criterion', ['gini', 'entropy']),
        'max_depth': hp.choice('dtree_max_depth', [None] + list(range(1, 10))),
        'min_samples_split': hp.randint('dtree_min_samples_split', 2, 10),
        'random_state': 24
    },
    {
        'type': 'rf',
        'n_estimators': hp.randint('rf_n_estimators', 20, 500),
        'max_features': hp.randint('rf_max_features', 2, 9),
        'criterion': hp.choice('rf_criterion', ['gini', 'entropy']),
        'random_state': 24
    },
    {
        'type': 'lr',
        'C': hp.loguniform('lr_C', np.log(0.01), np.log(10.0)),
        'random_state': 24
    }
])

In [17]:
algo = tpe.suggest
trials = Trials()
best_result = fmin(
        fn=objective, 
        space=search_space,
        algo=algo,
        max_evals=32,
        trials=trials)

  0%|          | 0/32 [00:00<?, ?trial/s, best loss=?]

Successfully registered model 'decision_tree_wine'.
2025/04/17 10:30:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: decision_tree_wine, version 1

Created version '1' of model 'decision_tree_wine'.
2025/04/17 10:30:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2025/04/17 10:30:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run unique-wolf-355 at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2/runs/3ea42d16afd0485db1bacbc43010b1ed.

2025/04/17 10:30:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2.



  3%|▎         | 1/32 [00:04<02:11,  4.23s/trial, best loss: -0.8837944664031621]

Registered model 'decision_tree_wine' already exists. Creating a new version of this model...
2025/04/17 10:30:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: decision_tree_wine, version 2

Created version '2' of model 'decision_tree_wine'.
2025/04/17 10:30:27 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2025/04/17 10:30:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run redolent-bird-293 at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2/runs/a3885b327caa4ea2a2c2c17153bd4a4f.

2025/04/17 10:30:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2.



  6%|▋         | 2/32 [00:07<01:52,  3.74s/trial, best loss: -0.8837944664031621]

...

 94%|█████████▍| 30/32 [01:35<00:06,  3.09s/trial, best loss: -0.9913043478260869]

Registered model 'random_forest_wine' already exists. Creating a new version of this model...
2025/04/17 10:31:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_wine, version 5

Created version '5' of model 'random_forest_wine'.
2025/04/17 10:31:59 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2025/04/17 10:31:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run mysterious-squid-387 at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2/runs/ae014de3122449e8a1de8af955b9508a.

2025/04/17 10:31:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2.



 97%|█████████▋| 31/32 [01:38<00:03,  3.12s/trial, best loss: -0.9913043478260869]

Registered model 'logistic_regression_wine' already exists. Creating a new version of this model...
2025/04/17 10:32:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_regression_wine, version 17

Created version '17' of model 'logistic_regression_wine'.
2025/04/17 10:32:02 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.

2025/04/17 10:32:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run loud-koi-409 at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2/runs/95ce6fe6884b4c2da13ae4aafe8908a8.

2025/04/17 10:32:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2.



100%|██████████| 32/32 [01:42<00:00,  3.19s/trial, best loss: -0.9913043478260869]


In [13]:
with mlflow.start_run(run_name="feature_selection"):
    rf_selector = RandomForestClassifier(n_estimators=100, random_state=24)
    rf_selector.fit(X_train_scaled, y_train)
    
    feature_importance = pd.DataFrame({
        'feature': X_train_scaled.columns,
        'importance': rf_selector.feature_importances_
    }).sort_values('importance', ascending=False)
    
    top_features = feature_importance.head(7)['feature'].tolist()

    X_train_selected = X_train_scaled[top_features]
    X_val_selected = X_val_scaled[top_features]
    X_test_selected = X_test_scaled[top_features]

    X_train_selected.to_parquet('save_data_lab2/x_train_selected.parquet')
    X_val_selected.to_parquet('save_data_lab2/x_val_selected.parquet')
    X_test_selected.to_parquet('save_data_lab2/x_test_selected.parquet')
    
    rf_selected = RandomForestClassifier(n_estimators=100, random_state=24)
    rf_selected.fit(X_train_selected, y_train)
    acc_selected = cross_val_score(rf_selected, X_train_selected, y_train, cv=5).mean()
    mlflow.log_metric('cv_accuracy_selected_features', acc_selected)
    
    mlflow.sklearn.log_model(
        rf_selector,
        artifact_path='feature_selector',
        registered_model_name='wine_feature_selector'
    )

    mlflow.sklearn.log_model(
        rf_selected,
        artifact_path='selected_features_model',
        registered_model_name='wine_classifier_selected_features'
    )
    
    mlflow.end_run()

Successfully registered model 'wine_feature_selector'.
2025/04/17 10:21:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine_feature_selector, version 1
Created version '1' of model 'wine_feature_selector'.
2025/04/17 10:21:33 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
Successfully registered model 'wine_classifier_selected_features'.
2025/04/17 10:21:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine_classifier_selected_features, version 1
Created version '1' of model 'wine_classifier_selected_features'.
2025/04/17 10:21:36 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/17 10:21:36 INFO mlflow.tracking._tracking

Top 3 models are random forest with max_features = 5 and n_estimators = 429, random forest model with max_features = 2 and n_estimators = 290, and random forest with max_features = 2 and n_estimators = 342.

In [14]:
with mlflow.start_run():
    mlflow.set_tags({
        "Model": "random-forest",
        "Data": "final-comprehensive-logging"
    })
    
    mlflow.log_params({
        "n_estimators": 429,
        "max_features": 5
    })
    
    rf = RandomForestClassifier(n_estimators=429, max_features=5, random_state=24)
    rf.fit(X_train_scaled, y_train)

    mlflow.log_artifact('save_data_lab2/x_train.parquet', 'training_data')
    mlflow.log_artifact('save_data_lab2/y_train.parquet', 'training_data')
    
    mlflow.log_artifact('save_data_lab2/x_val.parquet', 'validation_data')
    mlflow.log_artifact('save_data_lab2/y_val.parquet', 'validation_data')
    
    mlflow.log_artifact('save_data_lab2/x_test.parquet', 'test_data')
    mlflow.log_artifact('save_data_lab2/y_test.parquet', 'test_data')
    
    train_acc = accuracy_score(y_train, rf.predict(X_train_scaled))
    val_acc = accuracy_score(y_val, rf.predict(X_val_scaled))
    test_acc = accuracy_score(y_test, rf.predict(X_test_scaled))
    
    mlflow.log_metrics({
        'train_accuracy': train_acc,
        'validation_accuracy': val_acc,
        'test_accuracy': test_acc
    })
    
    mlflow.sklearn.log_model(
        rf, 
        artifact_path="random_forest_model",
        registered_model_name="wine_classifier_final"
    )
    
    print(f"Training Accuracy: {train_acc:.4f}")
    print(f"Validation Accuracy: {val_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")

mlflow.end_run()

Successfully registered model 'wine_classifier_final'.
2025/04/17 10:22:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: wine_classifier_final, version 1
Created version '1' of model 'wine_classifier_final'.
2025/04/17 10:22:14 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/17 10:22:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-stoat-140 at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2/runs/7a04b29aa37b4c208f09e4dcbcc2aa00.
2025/04/17 10:22:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlflow-service-982318063078.us-west2.run.app/#/experiments/2.


Training Accuracy: 1.0000
Validation Accuracy: 0.9655
Test Accuracy: 0.9444
